<a href="https://colab.research.google.com/github/tproffen/ORCSGirlsPython/blob/master/Numbers/ChaosAndFractals-Plotly.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hackathon
##### June 17, 2023 - 4:00pm until 4:02am

On June 17 four girls of the ORCSGirls Academy met at 4pm in Gather Town to embark on a journey into numbers and strange math things. Twelve hours later they emerged victoreously having conquered plotting, the logistics map, complex numbers and the Mandelbrot set.

In [1]:
import ipywidgets as widgets
import plotly.graph_objects as go
import numpy as np
from PIL import Image, ImageDraw
from IPython.display import YouTubeVideo

from google.colab import output


### Simple plotting

First we look at `matplotlib` and how to graph data. In this notebook we use tell matplotlib to create a widget (see line `%matplotlib widget` above) for the plot which allows us to update and zoom into plots. Note this might not work on Colaboratory - here you can use `%matplotlib inline` for non interactive plots.

While in other notebooks we have used `plt.plot()` to plot inline, this would add the data to the last widget. To create a new plot, we need to make a new figure as shown below.

In [ ]:
data=go.Scatter(x=[3,2,1.5,2.2,3.21,-2], name='Some data')

In [ ]:
output.enable_custom_widget_manager()
g = go.FigureWidget(data=[data])

In [ ]:
display(g)

FigureWidget({
    'data': [{'name': 'Some data',
              'type': 'scatter',
              'uid': 'b3fa6e95-5177-4e4e-84bd-ec5c87fd88e6',
              'x': [3, 2, 0, 2.2, 3.21, -2]}],
    'layout': {'autosize': True, 'template': '...'}
})

In [35]:
with g.batch_update():
  g.data[0].x = [3,2,0,2.2,7,-2]

One of the fun things we can do is to add more interactivity to the plot, e.g. using sliders for the slope and imtercept in an equation $y=mx+b$ for a line. See below.

In [2]:
def line(m,b):
    xvals=[]                        # Empty list for x values
    yvals=[]                        # Empty list for y values
    for x in np.arange(-4,4,0.05):  # Loop from -4 to 4 in steps of 0.05
        xvals.append(x)             # Add x vlaue to the list
        yvals.append(m*x+b)         # Calculate y and add to the list
    return xvals, yvals             # return both lists

Now we use that function to calculate the points and call it everytime the slider is moved. The code below will create sliders for $m$ and $b$ and allow the plot to be changed interactively :)

In [28]:
(x,y) = line(1,0)  # Initial line

# Making the graph
layout = go.Layout(title='Line Graph', yaxis_range=[-10,10], width=800, height=600)
data = go.Scatter(x=x, y=y, mode="lines", name='Line', xaxis='x', yaxis='y')
output.enable_custom_widget_manager()
graph = go.FigureWidget(data=[data], layout=layout)

# update routine
def update(change):
    (graph.data[0].x, graph.data[0].y) = line(mSlider.value, bSlider.value)

# Making the sliders
mSlider = widgets.FloatSlider(value=1.0, min=-5, max=5.0, step=0.1, description='Slope m:', continuous_update=True, readout=True, readout_format='.1f')
bSlider = widgets.FloatSlider(value=1.0, min=-5, max=5.0, step=0.1, description='Intercept b:', continuous_update=True, readout=True, readout_format='.1f')
mSlider.observe(update, names='value')
bSlider.observe(update, names='value')
sliders = widgets.HBox([mSlider,bSlider], layout=widgets.Layout(border='solid 2px #fff'))
display(widgets.VBox([sliders,graph]))


# Now the fun - the Logistic Map

Before we start coding this ourselves, here is a video that gives a good introduction on the equation and the many ways we encouter this.

In [ ]:
YouTubeVideo("ovJcsL7vyrk", width=600)

#### Here is the equation: $x_{n+1} = r x_{n}(1 - x_{n})$

Following the notation in the norebook, $r$ is the growth rate and $x$ is population. This is not the actual number, but the fraction of the maximum possible population, so it ranges from 0 to 1.

#### Step 1: Visualize the function

Let's define a function `growth` and plot how the population next year depends on the population this year;

In [ ]:
def growth(x,r):
    return r*x*(1-x)

In [ ]:
r = 2.0   # Growth rate of 2
x = 0.4   # Initial population 40% (or 0.4) of the possible maximum

xpts = np.arange(0.0,1.1,0.1)           # Values for x from 0 to 1 (note the 1.1)
ypts = [growth(x,r) for x in xpts]      # One line loop to calculate all the y's

fig, ax = plt.subplots(figsize=(6,4))   # Making the new plot
plt.xlabel('x_n')
plt.ylabel('x_n+1')

ax.plot(xpts,ypts)

Woohoo, just like in the video. He have a 'single hump function' with a negative feedback, meaning when the population gets too big the dowturn of the parabula will reduce the population as you can see on the graph.

#### Step 2: Behaviour as time goes on

We are most interested in the long term behaviour of the function as we repeatley call it. In other words what happens to the population after a long time. In the function below we calculate the value of $x_{n} = r x_{n-1}(1 - x_{n-1})$ for all $n$. Note that we have rewritten this so the value we calculate os $n$ and it depends on the earlier value $n-1$ which is how we implement it by using the list element `[-1]`.

In [ ]:
def growthtime(x0,r,nmax):
    vals=[x0]                                  # First element is the initial pop.
    for i in range(nmax-1):                    # Loop to the maximum number nmax we want.
                                               #   the -1 is because we already added x0
        vals.append(r*vals[-1]*(1-vals[-1]))   # Append the calculated vale
    return vals                                # Return all values

In [ ]:
fig, ax = plt.subplots(figsize=(6,4))
plt.xlabel('Time')
plt.ylabel('Population')

@widgets.interact(x0=(0.1,0.9,0.1),r=(0.5,4.0,0.01))
def update(x0=0.4, r=1.2):
    [l.remove() for l in ax.lines]
    ax.plot(growthtime(x0,r,100), color="blue", marker='o')    # Adding some markers

Explore how the function depends on `x0` and `r`. Anything spooky 😨

#### Step 3: Exploring as function of growth rate $r$

You might have seen strange behaviour for certain values of $r$, so next we explore the resulting populations after some time. Because there are multiple values the function oscillates around, we simply loop ofer $r$ and then plot the last `nplot` points ion a scatter plot.

In [ ]:
x0 = 0.4         # Starting population
nmax  = 300      # Maximum value of n for calculate
nplot = 150      # First point to plot, so we plot nplot:nmax

rvals=[]         # List for values of r
lvals=[]         # List for resulting values of x (there will be nmax values for each r)

for r in np.arange(0.9,4.0,0.001):              # Loop over r - note the fine grid
    lvals.append(growthtime(x0,r,nmax)[nplot:]) # Getting list of x starting at nplot
    rvals.append([r]*(nmax-nplot))              # We need the same number of r's so
                                                # lists are the same length

# Now we need to turn the list of lists into one contineous list for plotting
lvals = np.array(lvals).ravel()
rvals = np.array(rvals).ravel()

All that is left is to plot. Adjust marker size and color below. Also explore changing nmax, nplot and the step size in r. Enjoy.

In [ ]:
fig, ax = plt.subplots(figsize=(6,4))
plt.scatter(rvals,lvals,s=0.02,color='purple')

# Mandelbrot Set

Here is part two. The equation to generate the well known Mandelbrot set is simply $z_{n+1}=z_{n}^2 + c$ where $c$ is a complex number. Complex numbers have a real and imaginary part and can be visualised in 2D - think of the real and imaginary parts as x and y. These numbers solve the issue with square roots of negative numbers by defining $i=sqrt(-1)$ and a complex number is simply $a+ib$. Here $a$ is the real part and $b$ the imaginary one.

In Python we can create complex numbers with `c = complex(a,b)`.

#### Understanding the equation

Here is the equation again: $z_{n+1}=z_{n}^2 + c$. To see of a point $c$ in the colplex plain belongs to the Mandelbrot set, we iterate over the equation - measning we start with z=0 and plug it in to calculate the next z. Then we plug that one in to calculate the next and so on. If z stays a finite number even after infinite iterations, that point c we used is part of the set, otherwise it is not.

#### Creating a grey scale version

Let's define a function `mandelbrot`. We pass c to it and then keep evaluating the equation as long as the value of z stays below 2 (we picked that somewhat randomly) AND the number of iterations is still below the maximum.

So if the returned number of iterations is nmax, we know the value did not exceed 2, so the point c is part of the set. If n is less that nmax, the loop ended because the value was exceeding 2, so the point c is NOT part of the Mandelbrot set.

In [ ]:
nmax = 100

def mandelbrot(c):
    z=0
    n=0
    while abs(z)<= 2 and n < nmax:
        z = z*z + c
        n = n + 1

    return n

Now to make the beautiful images, we need to loop over x,y pixels of an image, convert the x,y to the values of the complex c and call the mandelbrot function. We then use the returned number of iterations n to set the color of the pixel.

In [ ]:
# Desired image size
width = 1000
height = 666

# The range in the complex plane to map onto the image
re_start = -2.0
re_end = 1.0
im_start = -1.0
im_end = 1.0

# Creating an empty image
im = Image.new('RGB', (width, height), (0,0,0))
draw = ImageDraw.Draw(im)

# Loop over all pixels
for x in range(width):
    for y in range(height):
        c = complex(re_start + (float(x) / width) * (re_end - re_start),   # Convert x,y to c
                    im_start + (float(y) / height) * (im_end - im_start))
        n = mandelbrot(c)
        color = int(n / nmax * 255)              # scaling the n to the range 0 to 255
        draw.point([x,y],(color,color,color))    # Using the value for r,g,b for greyscale

im   # show the image

#### Using colors

Above we used the value n to create a grey scale images using the color (n,n,n). It would be easy to make it other colors e.g. red by using (n,0,0) in the `draw.Point` command. To have colors changing as function of n, we can desacribe the colors as hue, saturation and value (color wheel) and use n to select the hue. Additionally we need to set the brightness to 0 is the point is part of the Mandelbrot set, so it is black. The code is very similar to before, just the assignment of color is different.

In [ ]:
# Desired image size
width = 1000
height = 666

# The range in the complex plane to map onto the image
re_start = -2.0
re_end = 1.0
im_start = -1.0
im_end = 1.0

# Creating an empty image
im = Image.new('HSV', (width, height), (0,0,0))
draw = ImageDraw.Draw(im)

for x in range(width):
    for y in range(height):
        c = complex(re_start + (float(x) / width) * (re_end - re_start),
                    im_start + (float(y) / height) * (im_end - im_start))
        n = mandelbrot(c)
        hue = int(n / nmax * 255)
        value = 255 if n < nmax else 0    # Here we make the point of the set black
        saturation = 255
        draw.point([x,y],(hue,saturation,value))

im=im.convert('RGB')   # We need to convert to RGB, so we can display it in the notebook
im

Time to save the image :)

In [ ]:
im.save('Beauty.png')

#### Saving using a colormap

In this case we create a grey scale image with just one number (0 to 255) per pixel. We can then use colormaps to translate the value to qa color when plotting.

Colormaps to try are: `'Accent', 'Accent_r', 'Blues', 'Blues_r', 'BrBG', 'BrBG_r', 'BuGn', 'BuGn_r', 'BuPu', 'BuPu_r', 'CMRmap', 'CMRmap_r', 'Dark2', 'Dark2_r', 'GnBu', 'GnBu_r', 'Greens', 'Greens_r', 'Greys', 'Greys_r', 'OrRd', 'OrRd_r', 'Oranges', 'Oranges_r', 'PRGn', 'PRGn_r', 'Paired', 'Paired_r', 'Pastel1', 'Pastel1_r', 'Pastel2', 'Pastel2_r', 'PiYG', 'PiYG_r', 'PuBu', 'PuBuGn', 'PuBuGn_r', 'PuBu_r', 'PuOr', 'PuOr_r', 'PuRd', 'PuRd_r', 'Purples', 'Purples_r', 'RdBu', 'RdBu_r', 'RdGy', 'RdGy_r', 'RdPu', 'RdPu_r', 'RdYlBu', 'RdYlBu_r', 'RdYlGn', 'RdYlGn_r', 'Reds', 'Reds_r', 'Set1', 'Set1_r', 'Set2', 'Set2_r', 'Set3', 'Set3_r', 'Spectral', 'Spectral_r', 'Wistia', 'Wistia_r', 'YlGn', 'YlGnBu', 'YlGnBu_r', 'YlGn_r', 'YlOrBr', 'YlOrBr_r', 'YlOrRd', 'YlOrRd_r', 'afmhot', 'afmhot_r', 'autumn', 'autumn_r', 'binary', 'binary_r', 'bone', 'bone_r', 'brg', 'brg_r', 'bwr', 'bwr_r', 'cividis', 'cividis_r', 'cool', 'cool_r', 'coolwarm', 'coolwarm_r', 'copper', 'copper_r', 'cubehelix', 'cubehelix_r', 'flag', 'flag_r', 'gist_earth', 'gist_earth_r', 'gist_gray', 'gist_gray_r', 'gist_heat', 'gist_heat_r', 'gist_ncar', 'gist_ncar_r', 'gist_rainbow', 'gist_rainbow_r', 'gist_stern', 'gist_stern_r', 'gist_yarg', 'gist_yarg_r', 'gnuplot', 'gnuplot2', 'gnuplot2_r', 'gnuplot_r', 'gray', 'gray_r', 'hot', 'hot_r', 'hsv', 'hsv_r', 'inferno', 'inferno_r', 'jet', 'jet_r', 'magma', 'magma_r', 'nipy_spectral', 'nipy_spectral_r', 'ocean', 'ocean_r', 'pink', 'pink_r', 'plasma', 'plasma_r', 'prism', 'prism_r', 'rainbow', 'rainbow_r', 'seismic', 'seismic_r', 'spring', 'spring_r', 'summer', 'summer_r', 'tab10', 'tab10_r', 'tab20', 'tab20_r', 'tab20b', 'tab20b_r', 'tab20c', 'tab20c_r', 'terrain', 'terrain_r', 'turbo', 'turbo_r', 'twilight', 'twilight_r', 'twilight_shifted', 'twilight_shifted_r', 'viridis', 'viridis_r', 'winter', 'winter_r'`

Most of the code is the same,except the type of image and the way we plot it.

In [ ]:
# Desired image size
width = 1000
height = 666

# The range in the complex plane to map onto the image
re_start = -2.0
re_end = 1.0
im_start = -1.0
im_end = 1.0

# Creating an empty image
im = Image.new('L', (width, height), 0)
draw = ImageDraw.Draw(im)

for x in range(width):
    for y in range(height):
        c = complex(re_start + (float(x) / width) * (re_end - re_start),
                    im_start + (float(y) / height) * (im_end - im_start))
        n = mandelbrot(c)
        value = int(n / nmax * 255)
        draw.point([x,y],value)

Note that we are using matplotlib to show the image now and get coordinate axes. We can specify the ranges for x and y as argument, so the coordinates are now the complex values rather than the pixel values.

In [ ]:
colormap = 'viridis'

fig, ax = plt.subplots(figsize=(6,4))
plt.xlabel('Real')
plt.ylabel('Imaginary')

ax.imshow(im, cmap=colormap, extent=[re_start,re_end,im_start,im_end])

Because we want to save the figure now and not just the image, the save command looks different.

In [ ]:
plt.savefig("Pretty.png")

#### Animation

Now we need to create a sequence of images we can then run as an animation zooming into an area. First let's just repeat the fucntion here, so we do not need to scroll when wanting to check things 😎

In [ ]:
nmax = 100

def mandelbrot(c):
    z=0
    n=0
    while abs(z)<= 2 and n < nmax:
        z = z*z + c
        n = n + 1

    return n

Now we need a function that creates the image. This is the same code as before just added to a function and we can pass the area in the complex plan we want to calculate.

In [ ]:
def mandelbrotImage(re_start,re_end,im_start,im_end,width,height):
    # Creating an empty image
    im = Image.new('L', (width, height))
    draw = ImageDraw.Draw(im)

    for x in range(width):
        for y in range(height):
            c = complex(re_start + (float(x) / width) * (re_end - re_start),
                        im_start + (float(y) / height) * (im_end - im_start))
            n = mandelbrot(c)
            value = int(n / nmax * 255)
            draw.point([x,y],value)

    return im

Next we need to create the frames for the animation. Here we pick a center spot and then zoom in by shrinking the ranges in x and y. We store the images in a list.

In [ ]:
# Desired image size
width = 600
height = 400

# Zoom factor
zoom = 0.8

# Number of frames (caution this gets slow)
nframes = 25

# Center for Zoom
re_center = -0.778
im_center = 0.125

re_start = re_center - 1.5
re_end = re_center + 1.5
im_start = im_center - 1.0
im_end = im_center + 1.0

# Empty list to store images
images=[]

for i in range(nframes):
    images.append(mandelbrotImage(re_start,re_end,im_start,im_end,width,height))
    print(f"Number {i} - Finished range {re_start:.5f}, {re_end:.5f} to {im_start:.5f}, {im_end:.5f}")

    re_range = 0.5 * abs(re_end - re_start)
    im_range = 0.5 * abs(im_end - im_start)

    re_start = re_center - zoom * re_range
    re_end   = re_center + zoom * re_range
    im_start = im_center - zoom * im_range
    im_end   = im_center + zoom * im_range

Now we turn the images into frames and assemble the animation.

In [ ]:
plt.close() # Cleaning up

frames = [] # for storing the generated images
fig, ax = plt.subplots(figsize=(6,4))

# Hide X and Y axes label marks
ax.xaxis.set_tick_params(labelbottom=False)
ax.yaxis.set_tick_params(labelleft=False)

# Hide X and Y axes tick marks
ax.set_xticks([])
ax.set_yticks([])

# Now loop over images to create the frames
for im in images:
    frames.append([plt.imshow(im, animated=True, cmap=colormap)])

ani = animation.ArtistAnimation(fig, frames, interval=500, blit=False, repeat_delay=0)
plt.show()

@widgets.interact(run=True)
def update(run):
    ani.pause() if not run else ani.resume()

ALl we have to do is save this wonderful movie ✨

In [ ]:
ani.save('mandelbrot.mp4')